# <font color='orange'>Catégorisez automatiquement des questions</font>

Demande :
- Un notebook de mise en oeuvre d’une approche non supervisée de proposition de mots clés, de type LDA avec visualisation en 2D des topics.
- En plus des mesures et scores techniques, mettre en place une méthode d’évaluation propre au contexte et identique pour chaque type de modélisation (du type taux de couverture des tags), avec une séparation du jeu de données pour l’évaluation.

##  <font color='navy'>Sommaire

**Partie 1 : Construction du Bag-of-words**
- <a href="http://localhost:8888/notebooks/OCR%20-%20IML/P5%20-%20Cat%C3%A9gorisez%20automatiquement%20des%20questions/Jouvin_Flavie_1_notebook_exploration_052024.ipynb?#C15#C1">Récupération des données</a>
- <a href="http://localhost:8888/notebooks/OCR%20-%20IML/P5%20-%20Cat%C3%A9gorisez%20automatiquement%20des%20questions/Jouvin_Flavie_1_notebook_exploration_052024.ipynb?#C15#C5">Nettoyage des données texte</a>
- <a href="http://localhost:8888/notebooks/OCR%20-%20IML/P5%20-%20Cat%C3%A9gorisez%20automatiquement%20des%20questions/Jouvin_Flavie_1_notebook_exploration_052024.ipynb?#C15#C10">Vectorisation du texte</a>
- <a href="http://localhost:8888/notebooks/OCR%20-%20IML/P5%20-%20Cat%C3%A9gorisez%20automatiquement%20des%20questions/Jouvin_Flavie_1_notebook_exploration_052024.ipynb?#C15#C15">Exploration des données</a>

**Partie 2 : Modelisation non supervisée**
- <a href="#C1">Préparation des données.</a>
    - <a href="#C11"> Transformation des données pour BoW</a>
    - <a href="#C12"> Transformation des données pour Tf-idf</a>
- <a href="#C5">LDA</a>
    - <a href="#C51"> LDA + BoW</a>
    - <a href="#C52"> LDA + Tf-idf</a>
    - <a href="#C53"> Visualisation avec LDAvis</a>

# <font color='navy'>PARTIE 2 - Modélisation non supervisée</font>

#### <font color='orange'>**Import des librairies python.**</font>

In [7]:
import pandas as pd
import numpy as np
import tqdm
import os
#import pickle
#from scipy import sparse

import nltk
nltk.download('punkt')

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models import LdaMulticore, CoherenceModel

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from pprint import pprint

#from sklearn.decomposition import LatentDirichletAllocation, NMF


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jouvi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


On dispose içi d'un problème de classification multilabel, ce qui signifie qu'une ligne de donnée peut-être associée à plusieurs étiquettes (contrairement à une classification classique où chaque échantillon est associé à un seul label).

## <a name="C1"><font color='navy'>Préparation des données.</font></a>

In [14]:
# Import des données.
df = pd.read_json('cleaned_df_final.json', orient='records', lines=True)

# Tokenise les chaînes en listes de mots.
data_lemmatized = df['sentence_bow_lem_pos'].apply(nltk.word_tokenize).tolist()

# Création du dictionnaire Gensim.
id2word = corpora.Dictionary(data_lemmatized)

# Filtre les tokens qui apparaissent dans moins de 5 documents et plus de 50% des documents.
id2word.filter_extremes(no_below=5, no_above=0.5)

#### <a name="C11"><font color='orange'>**Transformation des données pour BoW**</font></a>

In [16]:
# Conversion de chaque document en BoW (Bag-of-Words).
bow_corpus = [id2word.doc2bow(text) for text in data_lemmatized]

# Afficher la représentation BoW du premier document pour vérification
print("BoW du premier document :")
print(bow_corpus[0])

BoW du premier document :
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 4), (9, 2), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 3), (16, 1), (17, 3), (18, 1), (19, 10), (20, 4), (21, 2), (22, 1), (23, 2), (24, 1), (25, 2), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 2), (34, 1), (35, 4), (36, 1), (37, 2), (38, 1), (39, 2)]


In [17]:
print('Nombre de tokens: %d' % len(id2word))
print('Nombre de documents: %d' % len(bow_corpus))

Nombre de tokens: 1543
Nombre de documents: 2482


In [18]:
# Division en jeu d'entraînement et de test (70% entraînement, 30% test).
# bow_train_corpus, bow_test_corpus = train_test_split(bow_corpus, test_size=0.3, random_state=42)

# Division des index plutôt que des corpus.
train_indices, test_indices = train_test_split(range(len(bow_corpus)), test_size=0.3, random_state=42)

# Utilisation des index pour créer les jeux d'entraînement et de test.
bow_train_corpus = [bow_corpus[i] for i in train_indices]
bow_test_corpus = [bow_corpus[i] for i in test_indices]

In [19]:
print(f"Nombre de documents d'entraînement : {len(bow_train_corpus)}")
print(f"Nombre de documents de test : {len(bow_test_corpus)}")

Nombre de documents d'entraînement : 1737
Nombre de documents de test : 745


#### <a name="C12"><font color='orange'>**Transformation des données pour Tf-idf**</font></a>

In [21]:
# Convertion de chaque document en chaîne de caractères avant TF-IDF.
data_lemmatized_as_texts = [" ".join(text) for text in data_lemmatized]

# Instanciation du TF-IDF Vectorizer.
tfidf_vectorizer = TfidfVectorizer()

# Transformation des données en matrice TF-IDF.
#tfidf_matrix = tfidf_vectorizer.fit_transform(data_lemmatized_as_texts)

# Conversion en un format compatible avec Gensim.
#tfidf_corpus = gensim.matutils.Sparse2Corpus(tfidf_matrix, documents_columns=False)

# Création du vecteur TF-IDF.
tfidf_vectorizer = TfidfVectorizer(vocabulary=id2word.token2id)
tfidf_matrix = tfidf_vectorizer.fit_transform(data_lemmatized_as_texts)

# Conversion en corpus compatible avec Gensim.
tfidf_corpus = gensim.matutils.Sparse2Corpus(tfidf_matrix, documents_columns=False)

In [22]:
# Afficher la représentation TF-IDF du premier document.
print("TF-IDF du premier document :")
print(tfidf_corpus[0])

TF-IDF du premier document :
[(0, 0.07194389369446637), (1, 0.11281357452043805), (2, 0.06973608060525326), (3, 0.10437604646151229), (4, 0.14157172375060675), (5, 0.06800647164165864), (6, 0.08549182915212897), (7, 0.10385260847318084), (8, 0.32895063232884386), (10, 0.05581589529900243), (11, 0.08241520310312658), (12, 0.08937465558899674), (13, 0.09156207355839578), (14, 0.10437604646151229), (15, 0.1673049380418943), (16, 0.050379244554314744), (18, 0.0792834178257604), (19, 0.6094153323606673), (20, 0.24425913092189602), (21, 0.09767833066601546), (22, 0.09451104781340659), (23, 0.1865023715007644), (24, 0.11202322892786037), (25, 0.11041183257147799), (26, 0.12449763009656273), (27, 0.10235924171726139), (28, 0.10491344442928473), (29, 0.09656209424731911), (30, 0.11731019143824035), (31, 0.10913957098356082), (32, 0.07824216328166972), (33, 0.12364108482251779), (34, 0.11943333537130542), (35, 0.2591531916507775), (36, 0.08863280876716306), (37, 0.18902209562681319), (38, 0.0630

In [23]:
print('Nombre de tokens: %d' % len(id2word))
print('Nombre de documents: %d' % len(tfidf_corpus))

Nombre de tokens: 1543
Nombre de documents: 2482


In [24]:
# Division en jeu d'entraînement et de test (70% entraînement, 30% test).
# tfidf_train_corpus, tfidf_test_corpus = train_test_split(tfidf_corpus, test_size=0.3, random_state=42)

# Utilisation des index pour créer les jeux d'entraînement et de test.
tfidf_train_corpus = [tfidf_corpus[i] for i in train_indices]
tfidf_test_corpus = [tfidf_corpus[i] for i in test_indices]

In [25]:
print(f"Nombre de documents d'entraînement : {len(tfidf_train_corpus)}")
print(f"Nombre de documents de test : {len(tfidf_test_corpus)}")

Nombre de documents d'entraînement : 1737
Nombre de documents de test : 745


## <a name="C5"><font color='navy'>Latent Dirichlet Allocation (LDA)</font></a>

La LDA est un modèle non supervisé statistique qui permet de représenter des documents comme des mélanges de plusieurs sujets, et chaque sujet est lui-même un mélange de mots.

**Objectif du LDA :**

L'objectif de LDA est de déterminer à partir des documents existants :
- Les sujets présents dans l'ensemble des documents.
- La distribution de ces sujets dans chaque document.
- La distribution des mots dans chaque sujet.

**Comment ça marche :**

LDA commence par une hypothèse aléatoire sur la répartition des mots dans les sujets et des sujets dans les documents.
Ensuite, il ajuste ces hypothèses en fonction des mots qui apparaissent dans les documents réels.
Au fil de nombreuses itérations, le modèle affine ses hypothèses pour mieux correspondre aux données réelles.


C'est un outil puissant pour découvrir les thèmes dominants dans des collections de textes, comme des articles de recherche, des journaux, ou des messages sur les réseaux sociaux.

Nous utiliserons cette première approche pour détecter les topics associés aux mots de notre variable.

#### <a name="C51"><font color='orange'>**LDA + BoW**</font></a>

In [29]:
# Construction du modèle LDA.
def build_lda_model(corpus, dictionary, num_topics, alpha, eta=None):
    return gensim.models.LdaMulticore(corpus=corpus,
                                      id2word=dictionary,
                                      num_topics=num_topics,
                                      random_state=100,
                                      chunksize=100,
                                      passes=10,
                                      alpha=alpha,
                                      eta=eta)

- **Calcul du score de cohérence du modèle**

Le score de cohérence c_v est une métrique utilisée pour évaluer la cohérence des topics extraits dans les modèles de topic modeling. 

c_v mesure dans à quel point les mots d'un même topic apparaissent souvent ensemble dans les documents. Si un topic contient des mots qui apparaissent fréquemment dans les mêmes documents, ce topic est considéré comme cohérent.

**Comment fonctionne le score de cohérence c_v ?**

Le score de cohérence c_v utilise la probabilité conditionnelle pour mesurer la co-occurrence des mots dans les documents et les compare à une distribution empirique basée sur les corpus. 

Plus concrètement :
- Cohérence locale entre les mots : Le score de cohérence mesure la probabilité qu'un groupe de mots apparaisse ensemble dans le même document, ce qui est en lien avec les co-occurrences locales. Si deux mots souvent co-occurrences apparaissent fréquemment dans le même topic, leur probabilité de co-occurrence augmente, ce qui améliore la cohérence du topic.

- Cohérence globale : Ensuite, cette mesure est ajustée pour être plus robuste à des occurrences aléatoires dans les documents et peut tenir compte des mots qui n'ont pas de forte connexion sémantique entre eux.

Calcul : Le score c_v s'appuie sur des co-occurrences de paires de mots et utilise une mesure de similarité pour vérifier la cohérence entre les termes. 

Les étapes incluent :
- Le calcul de la probabilité de co-occurrence des mots dans le corpus.
- L'utilisation de la similarité de pointwise mutual information (PMI) entre les mots dans le corpus, qui mesure à quel point deux mots sont plus susceptibles de co-apparaître ensemble par rapport à leur occurrence individuelle.
- Une fonction de smoothing est souvent appliquée pour améliorer la mesure.
Le score c_v varie généralement de 0 à 1. Plus le score est élevé, plus les mots d'un topic sont cohérents et sémantiquement proches.

**Utilisation de la métrique c_v pour la recherche de tags.**
Dans le cadre d'un modèle de topic modeling, cette métrique semble pertinente. Dans notre cas elle permet d'extraire des groupes de mots censés être sémantiquement liés. Le score obtenu nous donne une information sur la qualité des tags extraits. Si les tags représentent des thèmes communs pour les questions, le score de cohérence sera une bonne métrique pour évaluer l'utilisé des topics extraite. 

- **Taux de couverture des tags** : mesure la proportion de documents pour lesquels le modèle a prédit au moins un tag exprimé en pourcentage du nombre total de documents.

- **Score de perplexité** : c'est une mesure utilisée pour évaluer la qualité d'un modèle de langage. Plus la perplexité est faible, meilleur est le modèle dans la prédiction des mots dans un corpus de texte.

In [74]:
# Calcul du score de cohérence.
def compute_coherence(corpus, dictionary, model, texts):
    coherence_model = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
    return coherence_model.get_coherence()

# Calcul du score de perplexité.
def calculate_perplexity(model, test_corpus):
    return model.log_perplexity(test_corpus)

# Calcul du taux de couverture des tags.
def calculate_tag_coverage(model, corpus):
    covered_docs = sum(1 for doc in corpus if len(model.get_document_topics(doc)) > 0)
    return (covered_docs / len(corpus)) * 100

**Recherche d'hyperparamètres**

Informations sur les paramètres :

**1. corpus :** Il s'agit de la représentation en Bag of Words (BoW) ou TF-IDF des documents (title + body). C'est une liste de documents où chaque document est représenté sous forme de liste de tuples (id_du_mot, fréquence_du_mot).<br>
**2. id2word :** C'est le dictionnaire où chaque mot unique est mappé à un identifiant dans le corpus.<br>
**3. num_topics :** Nombre de topics à extraire.<br>
**4. random_state :** Valeur de graine aléatoire pour rendre les résultats reproductibles.<br>
**5. chunksize :** Nombre de documents à traiter en même temps lors de l’entraînement du modèle. C'est une sorte de mini-batch.<br>
**6. passes :** Nombre de fois que le modèle doit passer sur l’ensemble du corpus lors de l’entraînement.<br>
**7. alpha :** Paramètre de dirichlet qui contrôle la distribution des topics dans chaque document. <br>
    - alpha = 'symmetric' : Tous les topics sont traités également, c’est-à-dire qu’il n’y a pas de préférence pour un topic spécifique.<br>
    - alpha = 'asymmetric' : Certains topics sont plus représentés dans certains documents que d'autres.<br>
    - alpha (float) : Valeur fixe pour contrôler la concentration des topics par document. Des valeurs plus élevées conduisent à moins de diversité de topics par document, et des valeurs plus faibles à plus de diversité.<br>
**8. eta :** Paramètre de dirichlet qui contrôle la distribution des mots dans chaque topic.<br>
    - eta = 'symmetric' : Tous les mots sont également susceptibles de faire partie de tous les topics.<br>
    - eta = 'asymmetric' : Certains mots sont plus représentés dans certains topics.<br>
    - eta (float) : Comme pour alpha, une valeur fixe peut être définie pour contrôler la concentration des mots dans chaque topic.<br>

In [78]:
# Vérification de l'existance d'un dossier 'results' / sinon le création du fichier.
if not os.path.exists('./results'):
    os.makedirs('./results')

In [80]:
# Rechercher des meilleurs hyperparamètres pour le modèle LDA.
def grid_search_lda(corpus_sets, dictionary, id2word, data_lemmatized, topics_range, alpha, eta):
    model_results = {'Validation_Set': [], 'Topics': [], 'Alpha': [], 'Eta': [], 'Coherence': []}
    
    pbar = tqdm.tqdm(total=(len(alpha) * len(eta) * len(topics_range) * len(corpus_sets)))
    
    for i, corpus_set in enumerate(corpus_sets):
        for k in topics_range:
            for a in alpha:
                for e in eta:
                    lda_model = build_lda_model(corpus_set, id2word, k, a, e if e != 'symmetric' else None)
                    coherence = compute_coherence(corpus_set, id2word, lda_model, data_lemmatized)
                    
                    # Enregistrer les résultats
                    model_results['Validation_Set'].append(f'Corpus {i+1}')
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Eta'].append(e)
                    model_results['Coherence'].append(coherence)
                    
                    pbar.update(1)
    
    pbar.close()
    
    return pd.DataFrame(model_results)

# Plage de topics.
min_topics = 5
max_topics = 15
topics_range = range(min_topics, max_topics)

# Alpha et Eta ranges (Beta a été enlevé)
alpha = list(np.arange(0.01, 1, 0.3)) + ['symmetric', 'asymmetric']
eta = list(np.arange(0.01, 1, 0.3)) + ['symmetric']

# Sets de validation (75% et 100% du corpus d'entraînement)
corpus_sets = [gensim.utils.ClippedCorpus(bow_train_corpus, int(len(bow_train_corpus) * 0.75)), bow_train_corpus]

# Lancer la recherche de grille pour les meilleurs paramètres sans beta
grid_results = grid_search_lda(corpus_sets, id2word, id2word, data_lemmatized, topics_range, alpha, eta)

# Sauvegarde des résultats dans un fichier CSV
grid_results.to_csv('./results/grid_lda_bow_results.csv', index=False)

# Charger les résultats et extraire les meilleurs hyperparamètres
best_result = grid_results.loc[grid_results['Coherence'].idxmax()]
print(f"Meilleurs paramètres trouvés :\n{best_result}")

100%|██████████████████████████████████████████████████████████████████████████████| 600/600 [2:41:29<00:00, 16.15s/it]

Meilleurs paramètres trouvés :
Validation_Set    Corpus 2
Topics                   7
Alpha                 0.91
Eta                   0.91
Coherence         0.481352
Name: 378, dtype: object


In [119]:
# Chemin vers le fichier CSV des résultats de la recherche de grille.
csv_file_path = './results/grid_lda_bow_results.csv'

# Chargement des résultats depuis le fichier CSV.
grid_results = pd.read_csv('./results/grid_lda_bow_results.csv')

# Extraction des meilleurs hyperparamètres.
best_result = grid_results.loc[grid_results['Coherence'].idxmax()]

# Affichage des meilleurs hyperparamètres.
print(f"Meilleurs paramètres trouvés :\n{best_result}")

bow_best_num_topics = int(best_result['Topics'])
bow_best_alpha = best_result['Alpha']
bow_best_eta = best_result['Eta']

# Vérification et conversion si nécessaire.
if isinstance(bow_best_alpha, str):
    bow_best_alpha = bow_best_alpha  # 'symmetric' ou 'asymmetric', pas de conversion nécessaire.
else:
    bow_best_alpha = float(bow_best_alpha)  # Conversion en float si c'est un nombre.

if isinstance(bow_best_eta, str):
    bow_best_eta = bow_best_eta  # 'symmetric', pas de conversion nécessaire.
else:
    bow_best_eta = float(bow_best_eta)  # Conversion en float si c'est un nombre.

Meilleurs paramètres trouvés :
Validation_Set              Corpus 2
Topics                             7
Alpha             0.9099999999999999
Eta               0.9099999999999999
Coherence                   0.481352
Name: 378, dtype: object


**Modèle final**

In [83]:
# Création du modèle final avec les meilleurs paramètres.
bow_best_num_topics = int(best_result['Topics'])
bow_best_alpha = best_result['Alpha']
bow_best_eta = best_result['Eta']

final_lda_model = build_lda_model(bow_train_corpus, id2word, bow_best_num_topics, bow_best_alpha, bow_best_eta)

In [149]:
# Évaluation du modèle final.

# Calcul et affichage du score de cohérence sur les données d'entraînement.
bow_lda_coherence_train = compute_coherence(bow_train_corpus, id2word, final_lda_model, data_lemmatized)
print(f"Score de cohérence sur les données d'entraînement : {bow_lda_coherence_train}")

# Calcul et affichage du score de perplexité sur les données de test.
bow_lda_perplexity_test = calculate_perplexity(final_lda_model, bow_test_corpus)
print(f"Score de perplexité sur les données de test : {bow_lda_perplexity_test}")

# Calcul et affichage du taux de couverture des tags sur les données de test.
bow_lda_coverage_test = calculate_tag_coverage(final_lda_model, bow_test_corpus)
print(f"Taux de couverture des tags sur les données de test : {bow_lda_coverage_test}%")

Score de cohérence sur les données d'entraînement : 0.479551149963347
Score de perplexité sur les données de test : -6.7707716583132065
Taux de couverture des tags sur les données de test : 100.0%


In [121]:
# Affichage des topics du modèle final
# pprint(final_lda_model.print_topics())

In [115]:
# Affichage des topics du modèle LDA.
topics = final_lda_model.print_topics()

# Fonction pour supprimer les poids et garder seulement les mots, avec le numéro de topic.
def display_topics_with_numbers(topics):
    clean_topics = []
    for topic in topics:
        topic_num = topic[0]  # Numéro du topic
        # Extraction de la liste de mots sans les coefficients.
        words = [word.split('*')[1].replace('"', '').strip() for word in topic[1].split(' + ')]
        clean_topic = f"Topic {topic_num}: " + ' '.join(words)
        clean_topics.append(clean_topic)
    
    return clean_topics

# Appel de la fonction pour formater et afficher les topics avec les numéros.
clean_topics = display_topics_with_numbers(topics)
pprint(clean_topics)

['Topic 0: test model image array code number data function lt import',
 'Topic 1: gt user const new return error function component service string',
 'Topic 2: int code template c text ltlt function return type main',
 'Topic 3: file error line info name server import docker command version',
 'Topic 4: class new public method void spring test context static child',
 'Topic 5: library path app project version gradle android source class file',
 'Topic 6: i string d null data var value return private type']


#### <a name="C52"><font color='orange'>**LDA + Tf-Idf**</font></a>

Utilisation de TF-IDF pour réduire l'impact des mots fréquents.

In [124]:
# Rechercher des meilleurs hyperparamètres pour le modèle LDA avec tf-Idf.
def grid_search_lda(corpus_sets, dictionary, id2word, data_lemmatized, topics_range, alpha, eta):
    model_results = {'Validation_Set': [], 'Topics': [], 'Alpha': [], 'Eta': [], 'Coherence': []}
    
    pbar = tqdm.tqdm(total=(len(alpha) * len(eta) * len(topics_range) * len(corpus_sets)))
    
    for i, corpus_set in enumerate(corpus_sets):
        for k in topics_range:
            for a in alpha:
                for e in eta:
                    lda_model = build_lda_model(corpus_set, id2word, k, a, e if e != 'symmetric' else None)
                    coherence = compute_coherence(corpus_set, id2word, lda_model, data_lemmatized)
                    
                    # Enregistrer les résultats
                    model_results['Validation_Set'].append(f'Corpus {i+1}')
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Eta'].append(e)
                    model_results['Coherence'].append(coherence)
                    
                    pbar.update(1)
    
    pbar.close()
    
    return pd.DataFrame(model_results)

# Plage de topics.
min_topics = 5
max_topics = 15
topics_range = range(min_topics, max_topics)

# Alpha et Eta ranges (Beta a été enlevé)
alpha = list(np.arange(0.01, 1, 0.3)) + ['symmetric', 'asymmetric']
eta = list(np.arange(0.01, 1, 0.3)) + ['symmetric']

# Sets de validation (75% et 100% du corpus d'entraînement)
corpus_sets = [gensim.utils.ClippedCorpus(tfidf_train_corpus, int(len(tfidf_train_corpus) * 0.75)), tfidf_train_corpus]

# Lancer la recherche de grille pour les meilleurs paramètres sans beta
grid_results = grid_search_lda(corpus_sets, id2word, id2word, data_lemmatized, topics_range, alpha, eta)

# Sauvegarde des résultats dans un fichier CSV
grid_results.to_csv('./results/grid_lda_tfidf_results.csv', index=False)

# Charger les résultats et extraire les meilleurs hyperparamètres
best_result = grid_results.loc[grid_results['Coherence'].idxmax()]
print(f"Meilleurs paramètres trouvés :\n{best_result}")

100%|██████████████████████████████████████████████████████████████████████████████| 600/600 [8:54:01<00:00, 53.40s/it]

Meilleurs paramètres trouvés :
Validation_Set      Corpus 2
Topics                     5
Alpha             asymmetric
Eta                     0.91
Coherence           0.638004
Name: 328, dtype: object


In [131]:
# Chemin vers le fichier CSV des résultats de la recherche de grille.
csv_file_path = './results/grid_lda_tfidf_results.csv'

# Chargement des résultats depuis le fichier CSV.
grid_results = pd.read_csv('./results/grid_lda_tfidf_results.csv')

# Extraction des meilleurs hyperparamètres.
best_result = grid_results.loc[grid_results['Coherence'].idxmax()]

# Affichage des meilleurs hyperparamètres.
print(f"Meilleurs paramètres trouvés :\n{best_result}")

tfidf_best_num_topics = int(best_result['Topics'])
tfidf_best_alpha = best_result['Alpha']
tfidf_best_eta = best_result['Eta']

# Vérification et conversion si nécessaire.
if isinstance(tfidf_best_alpha, str):
    tfidf_best_alpha = tfidf_best_alpha  # 'symmetric' ou 'asymmetric', pas de conversion nécessaire.
else:
    tfidf_best_alpha = float(tfidf_best_alpha)  # Conversion en float si c'est un nombre.

if isinstance(tfidf_best_eta, str):
    tfidf_best_eta = tfidf_best_eta  # 'symmetric', pas de conversion nécessaire.
else:
    tfidf_best_eta = float(tfidf_best_eta)  # Conversion en float si c'est un nombre.

Meilleurs paramètres trouvés :
Validation_Set              Corpus 2
Topics                             5
Alpha                     asymmetric
Eta               0.9099999999999999
Coherence                   0.638004
Name: 328, dtype: object


In [137]:
# Création du modèle final avec les meilleurs paramètres.
tfidf_best_num_topics = int(best_result['Topics'])
tfidf_best_alpha = best_result['Alpha']
tfidf_best_eta = float(best_result['Eta'])

final_lda_tfidf_model = build_lda_model(tfidf_train_corpus, id2word, tfidf_best_num_topics, tfidf_best_alpha, tfidf_best_eta)

In [138]:
# Évaluation du modèle final.

# Calcul et affichage du score de cohérence sur les données d'entraînement.
tfidf_lda_coherence_train = compute_coherence(tfidf_train_corpus, id2word, final_lda_tfidf_model , data_lemmatized)
print(f"Score de cohérence sur les données d'entraînement : {tfidf_lda_coherence_train}")

# Calcul et affichage du score de perplexité sur les données de test.
tfidf_lda_perplexity_test = calculate_perplexity(final_lda_tfidf_model , tfidf_test_corpus)
print(f"Score de perplexité sur les données de test : {tfidf_lda_perplexity_test}")

# Calcul et affichage du taux de couverture des tags sur les données de test.
tfidf_lda_coverage_test = calculate_tag_coverage(final_lda_tfidf_model , tfidf_test_corpus)
print(f"Taux de couverture des tags sur les données de test : {tfidf_lda_coverage_test}%")

Score de cohérence sur les données d'entraînement : 0.6323754193599684
Score de perplexité sur les données de test : -7.783402678648338
Taux de couverture des tags sur les données de test : 100.0%


In [141]:
# Affichage des topics du modèle LDA.
topics = final_lda_tfidf_model .print_topics()

# Fonction pour supprimer les poids et garder seulement les mots, avec le numéro de topic.
def display_topics_with_numbers(topics):
    clean_topics = []
    for topic in topics:
        topic_num = topic[0]  # Numéro du topic
        # Extraction de la liste de mots sans les coefficients.
        words = [word.split('*')[1].replace('"', '').strip() for word in topic[1].split(' + ')]
        clean_topic = f"Topic {topic_num}: " + ' '.join(words)
        clean_topics.append(clean_topic)
    
    return clean_topics

# Appel de la fonction pour formater et afficher les topics.
clean_topics = display_topics_with_numbers(topics)
pprint(clean_topics)

['Topic 0: gt error file new code app return class value data',
 'Topic 1: ng angularcompiler enable browsermodule angularcli zonejs '
 'androidlayoutwidthmatchparent bootstrap angularplatformbrowserdynamic '
 'appmodule',
 'Topic 2: ltdependencygt ltgroupidgtorgspringframeworkbootltgroupidgt '
 'ltexecutiongt ltgoalsgt ltconfigurationgt ltplugingt ltversiongtltversiongt '
 'ltpropertiesgt ltdependenciesgt ltpluginsgt',
 'Topic 3: pddataframe intent max double ltstringgtinclude environment num '
 'drop icon mac',
 'Topic 4: max intent pddataframe double ltstringgtinclude environment num '
 'drop icon mac']


**Comparaison des résultats**

In [151]:
# Synthèse des résultats LDA avec Bow et tf-Idf.
results = {
    'Metric': ['Coherence', 'Perplexity', 'Topic Coverage'],
    'LDA + BoW': [bow_lda_coherence_train, bow_lda_perplexity_test, bow_lda_coverage_test],
    'LDA + TF-IDF': [tfidf_lda_coherence_train, tfidf_lda_perplexity_test, tfidf_lda_coverage_test]
}

results_df = pd.DataFrame(results)
results_df

,Metric,LDA + BoW,LDA + TF-IDF
0,Coherence,0.479551,0.632375
1,Perplexity,-6.770772,-7.783403
2,Topic Coverage,100.000000,100.000000


#### <a name="C53"><font color='orange'>**Visualisation avec LDAVis.**</font></a>

In [155]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import gensim

In [167]:
# Préparation des données pour pyLDAvis.
lda_display = gensimvis.prepare(final_lda_model, tfidf_train_corpus, id2word, sort_topics=False)

In [169]:
# Visualisation avec pyLDAvis.
pyLDAvis.display(lda_display)

In [171]:
# Sauvegarde de la visualisation dans un fichier HTML.
pyLDAvis.save_html(lda_display, 'lda_visualization.html')

Thèmes principaux : 
- Topic 1 : Développement et évaluation de modèles et traitement de données.
- Topic 2 :Développement d'applications web et mobiles, gestion des états, interactions utilisateur, services et API, et programmation asynchrone. 
- Topic 3  Programmation système, gestion de la mémoire et optimisation des performances.:
- Topic 4  Administration des serveurs et des systèmes, développement logiciel, conteneurisation et gestion des environnements (DevOps).:
- Topic 5  Développement logiciel en Java, programmation orientée objet.:
- Topic 6  Développement d'applications mobiles et web.:
- Topic 7  Programmation orientée objet et manipulation de données.

Difficile d'interpréter les axes PCS1 et PCS2. Peut-être que ça peut nous aider à distinguer des actions plutôt Back-end/ Front-end mais ce n'est pas très précis.
hypothèse :
PCS2 en bas de l'axe correspond à tout ce qui représente du DevOps plutôt Back-end.
PCS2 en haut de l'axe on a plutôt des aspects Front-end.:

Limitation : LDA ne génère pas de tags explicites, il identifie des thématiques représentées par des groupes de mots. L'utilisation de modèles de classification supervisée pourraient être plus adaptés afin d'obtenir des tags plus spécifiques à la question posée.